# Exploratory Data Analysis

Tim Gormly
7/17/2024

In [11]:
import pandas as pd

df = pd.read_csv('Data/Gold/main.csv')

## Data Dimension

In [12]:
df.shape

(31937, 52)

## Dataframe contents

In [13]:
df.head()

,playerID,year,stint,tmID,pos,GP,G,A,Pts,PIM,...,SHG_per_game_score,SHA_per_game_score,GWG_per_game_score,SOG_per_game_score,award_count_per_game_score,+/-_score,stint_score,overall_score_vs_prev_stint,teammates_same_nationality,tsm_vs_prev_stint
0,aaltoan01,1997,1,ANA,C,3.0,0.0,0.0,0.0,0.0,...,0.5,0.5,0.000000,0.000000,0.5,0.857143,2.357143,NaN,NaN,NaN
1,aaltoan01,1998,1,ANA,C,73.0,3.0,5.0,8.0,24.0,...,0.5,0.5,0.000000,0.390664,0.5,0.071429,5.904306,NaN,NaN,NaN
2,aaltoan01,1999,1,ANA,C,63.0,7.0,11.0,18.0,26.0,...,0.5,0.5,0.999937,0.999999,0.5,0.000000,8.079260,NaN,NaN,NaN
3,aaltoan01,2000,1,ANA,C,12.0,1.0,1.0,2.0,2.0,...,0.5,0.5,0.000000,0.907407,0.5,1.000000,5.621846,NaN,NaN,NaN
4,abdelju01,2007,1,DET,L,2.0,0.0,0.0,0.0,2.0,...,0.5,0.0,0.000000,1.000000,0.5,0.423077,4.423075,NaN,NaN,NaN


## Check for missing values

In [14]:
df.isnull().sum()

playerID                           0
year                               0
stint                              0
tmID                               0
pos                                0
GP                                 0
G                                  0
A                                  0
Pts                                0
PIM                                0
+/-                                1
PPG                                0
PPA                                0
SHG                                0
SHA                                0
GWG                                0
SOG                                0
longest_stint                      0
award_count                        0
firstName                          0
lastName                           0
height                             0
weight                             0
birthYear                          0
birthCountry                       0
G_per_game                         0
A_per_game                         0
P

In [21]:
null_rows = df[df['stint_score'].isnull()]

for column_name, value in null_rows.loc[20868].items():
    print(f"{column_name}: {value}")

playerID: nedomva01
year: 1982
stint: 1
tmID: NYR
pos: R
GP: 1.0
G: 1.0
A: 0.0
Pts: 1.0
PIM: 0.0
+/-: nan
PPG: 0.0
PPA: 0.0
SHG: 0.0
SHA: 0.0
GWG: 0.0
SOG: 1.0
longest_stint: 0
award_count: 0.0
firstName: Vaclav
lastName: Nedomansky
height: 74.0
weight: 205.0
birthYear: 1944.0
birthCountry: Czech Republic
G_per_game: 1.0
A_per_game: 0.0
Pts_per_game: 1.0
PIM_per_game: 0.0
PPG_per_game: 0.0
PPA_per_game: 0.0
SHG_per_game: 0.0
SHA_per_game: 0.0
GWG_per_game: 0.0
SOG_per_game: 1.0
award_count_per_game: 0.0
G_per_game_score: 0.99999890000121
A_per_game_score: 0.0
Pts_per_game_score: 0.9999982380983424
PIM_per_game_score: 0.0
PPG_per_game_score: 0.0
PPA_per_game_score: 0.5
SHG_per_game_score: 0.5
SHA_per_game_score: 0.5
GWG_per_game_score: 0.0
SOG_per_game_score: 0.0
award_count_per_game_score: 0.5
+/-_score: nan
stint_score: nan
overall_score_vs_prev_stint: nan
teammates_same_nationality: nan
tsm_vs_prev_stint: nan
